In [1]:
import random
import operator
import numpy as np
import pandas as pd
import matplotlib.pylab as plt

In [2]:
class Address:
    def __init__(self, id_):
        self.id_ = id_
    
    def distance(self, address):
        return distanceMatrix[self.id_][address.id_]

    def __repr__(self):
        return str(self.id_)

In [3]:
class Fitness:
    def __init__(self, route):
        self.route = route
        self.distance = 0.0
        self.fitness = 0.0
    
    def routeDistance(self):
        if(self.distance == 0):
            pathDistance = 0
            for i in range(0, len(self.route)):
                fromAddress = self.route[i]
                toAddress = None
                if(i + 1 < len(self.route)):
                    toAddress = self.route[i + 1]
                else: 
                    toAddress = self.route[0]
                # print(str(fromAddress) + " " + str(toAddress) + " " + str(fromAddress.distance(toAddress)))
                pathDistance += fromAddress.distance(toAddress)
            self.distance = pathDistance
        return self.distance

    def routeFitness(self):
        if(self.fitness == 0):
            self.fitness = 1 / float(self.routeDistance())
        
        return self.fitness

In [11]:
def createRoute(addressList):
    route = random.sample(addressList, len(addressList))
    print(route)
    return route

def initialPopulation(populationSize, addressList):
    population = []
    for i in range(0, populationSize):
        population.append(createRoute(addressList))
    return population

def rankRoutes(population):
    fitnessResults = {}
    for i in range(0, len(population)):
        fitnessResults[i] = Fitness(population[i]).routeFitness()
    return sorted(fitnessResults.items(), key=operator.itemgetter(1), reverse=True)

def selection(populationRanked, eliteSize):
    selectionResults = []
    df = pd.DataFrame(np.array(populationRanked), columns=["Index", "Fitness"])
    df["cumulative_sum"] = df.Fitness.cumsum()
    df["cumulative_perc"] = 100 * df.cumulative_sum / df.Fitness.sum()

    for i in range(0, eliteSize):
        selectionResults.append(populationRanked[i][0])
    for i in range(0, len(populationRanked) - eliteSize):
        pick = 100 * random.random()
        for i in range(0, len(populationRanked)):
            if(pick <= df.iat[i, 3]):
                selectionResults.append(populationRanked[i][0])
                break
    return selectionResults

def matingPool(population, selectionResults):
    matingpool = []
    for i in range(0, len(selectionResults)):
        index = selectionResults[i]
        matingpool.append(population[index])
    return matingpool

def breed(parent1, parent2):
    child = []
    childP1 = []
    childP2 = []

    geneA = int(random.random() * len(parent1))
    geneB = int(random.random() * len(parent2))

    startGene = min(geneA, geneB)
    endGene = max(geneA, geneB)

    for i in range(startGene, endGene):
        childP1.append(parent1[i])

    childP2 = [item for item in parent2 if item not in childP1]

    child = childP1 + childP2
    return child

def breedPopulation(matingpool, eliteSize):
    children = []
    length = len(matingpool) - eliteSize
    pool = random.sample(matingpool, len(matingpool))

    for i in range(0, eliteSize):
        children.append(matingpool[i])

    for i in range(0, length):
        child = breed(pool[i], pool[len(matingpool) - i - 1])
        children.append(child)
    return children

def mutate(individual, mutationRate):
    for swapped in range(len(individual)):
        if(random.random() < mutationRate):
            swapWith = int(random.random() * len(individual))

            address1 = individual[swapped]
            address2 = individual[swapWith]

            individual[swapped] = address2
            individual[swapWith] = address1

    return individual

def mutatePopulation(population, mutationRate):
    mutatedPopulation = []

    for individual in range(0, len(population)):
        mutatedIndividual = mutate(population[individual], mutationRate)
        mutatedPopulation.append(mutatedIndividual)
    return mutatedPopulation

def nextGeneration(currentGen, eliteSize, mutationRate):
    popRanked = rankRoutes(currentGen)
    selectionResults = selection(popRanked, eliteSize)
    matingpool = matingPool(currentGen, selectionResults)
    children = breedPopulation(matingpool, eliteSize)
    nextGeneration = mutatePopulation(children, mutationRate)
    return nextGeneration

def geneticAlgorithm(population, popSize, eliteSize, mutationRate, generations):
    progress = []
    pop = initialPopulation(popSize, population)
    progress.append(1 / rankRoutes(pop)[0][1])
    print("Initial distance: " + str(1 / rankRoutes(pop)[0][1]))

    for i in range(0, generations):
        pop = nextGeneration(pop, eliteSize, mutationRate)
        progress.append(1 / rankRoutes(pop)[0][1])

    print("Final distance: " + str(1 / rankRoutes(pop)[0][1]))
    bestRouteIndex = rankRoutes(pop)[0][0]
    bestRoute = pop[bestRouteIndex]
    print(bestRoute)

    # plt.plot(progress)
    # plt.ylabel('Distance')
    # plt.xlabel('Generation')
    # plt.show()

In [15]:
# distanceMatrix = [[0,       2,      np.inf, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf],
#                   [2,       0,      3,      np.inf, np.inf, np.inf, np.inf, np.inf, np.inf],
#                   [np.inf,  3,      0,      3     , np.inf, np.inf, 1     ,      2,      1],
#                   [np.inf,  np.inf, 3,      0     , 2     , 2     , np.inf, np.inf, np.inf],
#                   [np.inf,  np.inf, np.inf, 2     , 0     , 2     , np.inf, np.inf, np.inf],
#                   [np.inf,  np.inf, np.inf, 2     , 2     , 0     , np.inf, np.inf, np.inf],
#                   [np.inf,  np.inf, 1     , np.inf, np.inf, np.inf, 0,      np.inf, np.inf],
#                   [np.inf,  np.inf, 2     , np.inf, np.inf, np.inf, np.inf, 0,      np.inf],
#                   [np.inf,  np.inf, 1     , np.inf, np.inf, np.inf, np.inf, np.inf, 0     ]]
                  
# crossroadList = []


# geneticAlgorithm(population=crossroadList, popSize=100, eliteSize=20, mutationRate=0.01, generations=200)

In [13]:
distanceMatrix = np.full((403, 403), np.inf)

flag = False
crossroadList = []
with open('../data/graphs/tolmin.net', 'r', encoding="utf-8") as f:
    for line in f.readlines():
        if(line.strip().endswith("crossroad")):
            vals = line.strip().split(" ")
            crossroadList.append(Address(int(vals[0]) - 1))
        elif(line.startswith("*edges")):
            flag = True
            continue
        if(flag):
            vals = line.strip().split(" ")
            node1 = vals[0]
            node2 = vals[1]
            dist = vals[4]
            # print(node1 + " " + node2 + " " + dist)
            if(float(dist) != 0.0):
                distanceMatrix[int(node1) - 1][int(node2) - 1] = float(dist)
                distanceMatrix[int(node2) - 1][int(node1) - 1] = float(dist)
for i in range(0, len(distanceMatrix)):
    distanceMatrix[i][i] = 0

In [14]:
geneticAlgorithm(population=crossroadList, popSize=100, eliteSize=20, mutationRate=0.01, generations=200)

360, 367, 377, 388, 386, 335, 379, 384, 352, 345, 375, 342, 347, 385, 341, 333, 395, 366, 374, 350, 353, 328, 376, 389, 354, 361, 359, 393, 329, 387, 396, 381, 368, 372, 346, 324, 338, 402, 337, 358]
[328, 367, 361, 336, 344, 338, 393, 374, 391, 394, 354, 399, 368, 369, 359, 376, 325, 396, 363, 381, 350, 375, 349, 348, 335, 400, 388, 402, 326, 339, 372, 395, 342, 384, 390, 346, 379, 358, 397, 337, 392, 345, 360, 333, 373, 347, 389, 329, 355, 327, 334, 351, 380, 356, 343, 362, 385, 340, 364, 383, 386, 353, 332, 352, 398, 382, 378, 365, 366, 331, 330, 377, 387, 357, 370, 401, 324, 341, 371]
[385, 343, 347, 372, 332, 394, 363, 381, 387, 335, 350, 378, 395, 325, 334, 370, 368, 348, 384, 369, 338, 397, 375, 401, 392, 374, 402, 398, 341, 331, 354, 344, 377, 388, 333, 361, 337, 357, 355, 339, 393, 349, 386, 364, 379, 326, 336, 399, 346, 329, 366, 380, 352, 351, 390, 353, 342, 328, 373, 345, 371, 362, 340, 358, 400, 367, 383, 396, 330, 365, 356, 324, 376, 359, 382, 389, 327, 360, 391]
[341, 38

ZeroDivisionError: float division by zero